***GENERATED CODE FOR regmanual PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Name', 'transformation_label': 'String Indexer'}], 'feature': 'Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '007: Tomorrow Never Dies', 'max': 'wwe Smackdown vs. Raw 2006', 'missing': '0', 'distinct': '431'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Name'}, {'feature_label': 'Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Platform', 'transformation_label': 'String Indexer'}], 'feature': 'Platform', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '2600.0', 'stddev': '0.0', 'min': '2600', 'max': 'XOne', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Platform'}, {'feature_label': 'Platform', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Platform')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Year_of_Release', 'transformation_label': 'String Indexer'}], 'feature': 'Year_of_Release', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '2005.16', 'stddev': '7.31', 'min': '1980', 'max': 'N/A', 'missing': '0', 'distinct': '38'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Year_of_Release'}, {'feature_label': 'Year_of_Release', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Year_of_Release')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Genre', 'transformation_label': 'String Indexer'}], 'feature': 'Genre', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Action', 'max': 'Strategy', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Genre'}, {'feature_label': 'Genre', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Genre')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Publisher', 'transformation_label': 'String Indexer'}], 'feature': 'Publisher', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '505 Games', 'max': 'Warner Bros. Interactive Entertainment', 'missing': '0', 'distinct': '34'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Publisher'}, {'feature_label': 'Publisher', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Publisher')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'User_Score', 'transformation_label': 'String Indexer'}], 'feature': 'User_Score', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '355', 'mean': '7.68', 'stddev': '1.29', 'min': '2.6', 'max': 'tbd', 'missing': '145', 'distinct': '58'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'User_Score'}, {'feature_label': 'User_Score', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('User_Score')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Developer', 'transformation_label': 'String Indexer'}], 'feature': 'Developer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '357', 'mean': '', 'stddev': '', 'min': '1st Playable Productions', 'max': 'Zipper Interactive', 'missing': '143', 'distinct': '129'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Developer'}, {'feature_label': 'Developer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Developer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Rating', 'transformation_label': 'String Indexer'}], 'feature': 'Rating', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '357', 'mean': '', 'stddev': '', 'min': 'E', 'max': 'T', 'missing': '143', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Rating'}, {'feature_label': 'Rating', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Rating')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pyspark


def SVMRegressor(sparkDF, features, labels, TrainingPercent=100):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    clf = SVR()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    clf.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % clf.score(X_test, y_test))

    data = {'model': clf,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run regmanualHooks.ipynb
try:
	#sourcePreExecutionHook()

	videogamesalehitoryeqlf = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1715327213046_Video_Games_Sales_History.csv', 'filename': 'Video_Games_Sales_History.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/DNM_TESTING 9th May/Test data/Video_Games_Sales_History.csv', 'viewFileName': 'Video_Games_Sales_History.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regmanualHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(videogamesalehitoryeqlf,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Name", "transformation_label": "String Indexer"}], "feature": "Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "007: Tomorrow Never Dies", "max": "wwe Smackdown vs. Raw 2006", "missing": "0", "distinct": "431"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Name"}, {"transformationsData": [{"feature_label": "Platform", "transformation_label": "String Indexer"}], "feature": "Platform", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "2600.0", "stddev": "0.0", "min": "2600", "max": "XOne", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Platform"}, {"transformationsData": [{"feature_label": "Year_of_Release", "transformation_label": "String Indexer"}], "feature": "Year_of_Release", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "2005.16", "stddev": "7.31", "min": "1980", "max": "N/A", "missing": "0", "distinct": "38"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Year_of_Release"}, {"transformationsData": [{"feature_label": "Genre", "transformation_label": "String Indexer"}], "feature": "Genre", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Action", "max": "Strategy", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Genre"}, {"transformationsData": [{"feature_label": "Publisher", "transformation_label": "String Indexer"}], "feature": "Publisher", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "505 Games", "max": "Warner Bros. Interactive Entertainment", "missing": "0", "distinct": "34"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Publisher"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NA_Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.02", "stddev": "3.4", "min": "0.0", "max": "41.36", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "NA_Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EU_Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.92", "stddev": "2.06", "min": "0.0", "max": "28.96", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "EU_Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "JP_Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.82", "stddev": "1.32", "min": "0.0", "max": "10.22", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "JP_Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Other_Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.6", "stddev": "0.84", "min": "0.0", "max": "10.57", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Other_Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Global_Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6.36", "stddev": "6.12", "min": "2.89", "max": "82.53", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Global_Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Critic_Score", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "349", "mean": "84.38", "stddev": "9.0", "min": "49", "max": "98", "missing": "151"}, "updatedLabel": "Critic_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Critic_Count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "349", "mean": "51.25", "stddev": "25.98", "min": "7", "max": "113", "missing": "151"}, "updatedLabel": "Critic_Count"}, {"transformationsData": [{"feature_label": "User_Score", "transformation_label": "String Indexer"}], "feature": "User_Score", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "355", "mean": "7.68", "stddev": "1.29", "min": "2.6", "max": "tbd", "missing": "145", "distinct": "58"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "User_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "User_Count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "353", "mean": "956.14", "stddev": "1462.72", "min": "4", "max": "10179", "missing": "147"}, "updatedLabel": "User_Count"}, {"transformationsData": [{"feature_label": "Developer", "transformation_label": "String Indexer"}], "feature": "Developer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "357", "mean": "", "stddev": "", "min": "1st Playable Productions", "max": "Zipper Interactive", "missing": "143", "distinct": "129"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Developer"}, {"transformationsData": [{"feature_label": "Rating", "transformation_label": "String Indexer"}], "feature": "Rating", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "357", "mean": "", "stddev": "", "min": "E", "max": "T", "missing": "143", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Rating"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regmanualHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=SVMRegressor(autofe, ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Critic_Score", "Critic_Count", "User_Count", "Name_stringindexer", "Platform_stringindexer", "Year_of_Release_stringindexer", "Genre_stringindexer", "Publisher_stringindexer", "User_Score_stringindexer", "Developer_stringindexer", "Rating_stringindexer"], "Global_Sales")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
